In [1]:
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_checker import check_env
from env import DangerousDaveEnv
import time, os
from custom_cnn import policy_kwargs
import torch
from stable_baselines3.common.vec_env import SubprocVecEnv

In [4]:
# Setting up the device
device = "mps" if torch.backends.mps.is_available() else "cpu"
device = torch.device(device)

# Manual assignment of arguments (replace with your desired values or use ipywidgets for interactivity)
train = False  # equivalent to --train in argparse
evaluate = True  # equivalent to --evaluate in argparse
model_name = "ppo_test"  # manually specify or generate a name
env_rep_type = 'text'  # 'text' or 'image'
model_type = 'PPO'  # 'DQN', 'RND', or 'PPO'
retrain = False  # equivalent to --retrain in argparse

# Your existing logic below
checkpoint_timestamp = int(time.time())
if not model_name:
    model_name = "checkpoints/dqn_ddave_{}".format(checkpoint_timestamp)

tensorboard_log = f"tensorboard_log/{model_name}"
tensorboard_log_run_name = '0'
print(model_name,tensorboard_log)
# Create the DangerousDaveEnv environment
env = DangerousDaveEnv(render_mode="human", env_rep_type=env_rep_type)


total_timesteps=100000

ppo_test tensorboard_log/ppo_test
(1, 11, 19)
Box(0, 255, (1, 11, 19), uint8)


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP:

In [6]:
if model_type == 'DQN':
    if train:
        # Define and train the DQN agent
        if retrain:
            model = DQN.load("checkpoints/{}".format(model_name),tensorboard_log=tensorboard_log)
            model.exploration_fraction  = 0.7
            model.set_env(env)
        else:
            model = DQN("CnnPolicy", env, verbose=1, batch_size=256, policy_kwargs=policy_kwargs,
                        learning_starts=1000, exploration_fraction=0.5, exploration_final_eps=0.01, device=device,
                        target_update_interval=5000, buffer_size=100000,tensorboard_log=tensorboard_log)

        model.learn(total_timesteps=total_timesteps, progress_bar=True,tb_log_name=tensorboard_log_run_name,log_interval=1)
        
        # Save the trained model if desired
        model.save("checkpoints/{}".format(model_name))

    if evaluate:
        # Evaluate the trained model
        model = DQN.load("checkpoints/{}".format(model_name), env=env,tensorboard_log=tensorboard_log)

elif model_type == 'PPO':
    if train:
        # Define and train the PPO agent
        if retrain:
            model = PPO.load("checkpoints/{}".format(model_name), env=env,tensorboard_log=tensorboard_log)
        else:
            model = PPO("CnnPolicy", env, verbose=1, batch_size=256, policy_kwargs=policy_kwargs, device=device,
                        tensorboard_log=tensorboard_log,ent_coef=0.01,vf_coef=1)

        model.learn(total_timesteps=total_timesteps, progress_bar=True,tb_log_name=tensorboard_log_run_name,log_interval=1)
        
        # Save the trained model if desired
        model.save("checkpoints/{}".format(model_name))

    if evaluate:
        # Evaluate the trained model
        model = PPO.load("checkpoints/{}".format(model_name), env=env,tensorboard_log=tensorboard_log)

if evaluate:
    obs, info = env.reset()
    terminated = False
    truncated = False
    while not (terminated or truncated):
        action, _ = model.predict(obs, deterministic=True)
        obs, rewards, terminated, truncated, info = env.step(action)
        env.render()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


KeyboardInterrupt: 

In [ ]:
logfile = tensorboard_log+'/'+tensorboard_log_run_name +'_5/'
logfile

In [ ]:
!tensorboard --logdir {logfile}